In [98]:
import tensorflow as tf
import numpy as np

from collections import namedtuple
from tqdm import tqdm
import os
import time

In [99]:
# Reading the data
data = open('sample_input.txt').read()

# Vocabulary business

# 1. Create a list of unique characters
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)

char_to_ix = {ch:i for i,ch in enumerate(chars) }
ix_to_chars = {i:ch for i,ch in enumerate(chars)}

x=np.zeros(len(data))
for i, c in enumerate(data):
    x[i]=char_to_ix[c]

data=x

print("The dataset has %d characters and %d unique." %(data_size, vocab_size))

The dataset has 5283795 characters and 80 unique.


In [143]:
# Defining hyperparameter tuple and setting hyper parameters
hparams = namedtuple('hyper_parameters', 
                     'hidden_size, seq_length, learning_rate,'
                     'batch_size, vocab_size,'
                    'num_epochs, num_layers, keep_prob')


# Using the hyper parameters also used by:
# Martin Gorner
#https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/rnn_train.py

hps = hparams(hidden_size=512,
              seq_length=30,
              learning_rate=1e-3,
              batch_size=200,
              vocab_size=vocab_size,
              num_epochs=50,
             num_layers=3,
             keep_prob=0.8)

In [144]:
class babble(object):
    def __init__(self, hps, mode, debug=True):
        self.hps=hps
        self.mode=mode
        self.batch_pointer=None
    
        
    def buildGraph(self, variant="fixed_length"):
        # Extracting structural specifics from HPS
        D = self.hps.vocab_size
        H = self.hps.hidden_size
        N = self.hps.batch_size
        T = self.hps.seq_length

        # Placeholder
        with tf.name_scope("PlaceHolders"):
            self.X = tf.placeholder(tf.int32, [None, None], "Inputs")
            self.Y = tf.placeholder(tf.int32, [None, None], "Expected_Output")
            #is_training = tf.placeholder(tf.bool)
            self.h0 = tf.placeholder(tf.float32, [None, self.hps.num_layers*self.hps.hidden_size], "initial_hidden_state")
        
        # No projection to embedding is performed in this experiment
        # Inputs are simply translated to one hot
        inputs = tf.one_hot(self.X,depth=self.hps.vocab_size)
            
        #with tf.name_scope("batch_norm"):
        #    inputs = tf.layers.batch_normalization(inputs)
        cell = tf.contrib.rnn.GRUCell(H)
        
    
        cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=self.hps.keep_prob)
        

        multi_cell = tf.contrib.rnn.MultiRNNCell([cell]*self.hps.num_layers, state_is_tuple=False)

        input_shape = tf.shape(self.X)
        
        #states = multi_cell.zero_state(self.hps.batch_size, tf.float32)
        self.zerostate = multi_cell.zero_state(input_shape[0], dtype=tf.float32) 
        outputs, self.hidden_state = tf.nn.dynamic_rnn(
                                         cell=multi_cell,
                                         dtype=tf.float32,
                                         inputs=inputs,
                                         initial_state=self.h0
                     
        )

        
        # Add an operation to update the train states with the last state tensors.
        #update_op = get_state_update_op(states, last_states)


    
        #with tf.control_dependencies([states.assign(last_states)]):
        #    outputs = outputs

             
        with tf.name_scope("Dense_Output_Layer"):
            outputs=tf.reshape(outputs, [-1, H])
            scores=tf.layers.dense(outputs, D)
        tf.summary.histogram('scores', scores)
        self.scores=scores
        return self.scores
      
        
    def trainStep(self, scores):
        # Extracting structural specifics from HPS
        D = self.hps.vocab_size
        H = self.hps.hidden_size
        N = self.hps.batch_size
        T = self.hps.seq_length
        
      
        
        # Define the additional part of the network Used for training
        # Loss and Optimizer
        y_int=tf.reshape(self.Y, [-1])
        with tf.name_scope("COST"):
            loss = tf.nn.softmax_cross_entropy_with_logits(
                labels=tf.one_hot(y_int,self.hps.vocab_size),
                logits=scores,
                name="softMaxCrossEntropy"
            )
            loss = tf.reduce_mean(loss)
        tf.summary.scalar('loss_', loss)
        
        
        with tf.name_scope("Predictions"):
            predictions = tf.cast(tf.argmax(scores, axis=-1, name="predictions"), tf.int32)
        #accuracy=tf.reduce_mean(tf.cast(tf.equal(predictions, self.Y), tf.float32))
                                         
        solver = tf.train.AdamOptimizer(self.hps.learning_rate)
        #solver = tf.train.GradientDescentOptimizer(hps.learning_rate)
        #solver = tf.train.MomentumOptimizer(hps.learning_rate, 0.9)
        
        tvars  = tf.trainable_variables()
        gs_int = tf.gradients(loss, tvars)
        grads = list(zip(gs_int, tf.trainable_variables()))
        gs, _  = tf.clip_by_global_norm(gs_int, 3.0)
        # for grad, var in grads:
        #     tf.summary.histogram(var.name + '/gradient', grad)
        #tf.summary.histogram('gradients', gs)
        train_step = solver.apply_gradients(zip(gs,tvars), global_step=tf.contrib.framework.get_or_create_global_step())
        #train_step = solver.apply_gradients(grads, global_step=tf.contrib.framework.get_or_create_global_step())
        summary = tf.summary.merge_all()
        return train_step, loss, summary, self.zerostate
   
    def train(self,sess, data,train_ops, writer=None, print_every=10):
        itr=0
        num_batches=data.shape[0]//self.hps.batch_size//self.hps.seq_length
        #print(num_batches)
        #istate = sess.run(self.zerostate) # initial zero input state (a tuple)
        for e in tqdm(list(range(hps.num_epochs)), desc='epoch'):
            
            total_correct=0
            #print("Reset", total_correct)
            for i in range(num_batches):
                itr+=1
                x,y = self.getNextBatch(data)
                feed_dict={self.X: x, self.Y:y, self.h0:np.zeros((self.hps.batch_size, self.hps.hidden_size*self.hps.num_layers))}
                _, loss, summary,_ = sess.run(train_ops, feed_dict=feed_dict)
                #if (i % print_every == 0):
                #print("Loss, Iter %d: %f" %(i, loss))
                if writer is not None:
                    writer.add_summary(summary)
            #Sample after every 10 epochs to see how we are doing
            if (itr%200 == 0): 
                self.sample(sess,self.scores)

        
    
    def sample(self, sess, scores, 
               seed="I have something to say",
               length=200, beam_width=5):
        #istate = sess.run(self.zerostate) # initial zero input state (a tuple)
        pd = tf.nn.softmax(scores)
        x= [char_to_ix[i] for i in seed]
        x=np.asarray(x).reshape(1,-1)
        y=np.zeros_like(x)
        feed_dict={self.X:x, self.Y:y, self.h0:np.zeros((1,self.hps.hidden_size*self.hps.num_layers))}
        pred_str=[]
        #print(x.shape, y.shape)
        for i in range(length):
            p,h0=sess.run([pd,self.hidden_state], feed_dict=feed_dict)
            feed_dict[self.h0]=h0
            p=p[-1]
            #print(p.shape)
            ix = np.random.choice(range(self.hps.vocab_size), p=p.ravel())
            #ix=np.argmax(p.ravel())
            p.reshape(1,-1)
            feed_dict[self.X]=ix*np.ones((1,1))
            pred_str.append(ix_to_chars[ix])
        txt=''.join(pred_str)
        #print(len(pred_str))
        print ('----\n %s \n----' % (txt, ))

    # data is expected to be numpy array of indices
    def getNextBatch(self, data):
        #print(data.shape[0])
        if self.batch_pointer is None:
            segment=data.shape[0]//self.hps.batch_size
            self.batch_pointer = np.array([offset*segment for offset in range(self.hps.batch_size)])
        else:
            self.batch_pointer += 1
            self.batch_pointer %= data.shape[0]
        
        
        x=np.zeros((self.hps.batch_size, self.hps.seq_length))
        y=np.zeros((self.hps.batch_size, self.hps.seq_length))
        
        indices=self.batch_pointer
        
        for i in range(self.hps.seq_length):
            x[:,i]=np.take(data,indices, mode='wrap')
            y[:,i]=np.take(data,indices+1, mode='wrap')
            indices+=1
            
        return x,y
    
#def batch2String(x):
        
        
                                
        




In [145]:
model=babble (hps,'train')
sample=model.getNextBatch(data)
print(sample[0].shape)



(200, 30)


In [146]:
tf.reset_default_graph()
scores=model.buildGraph()
train_ops=model.trainStep(scores)

config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
sess=tf.Session(config=config)
summaries=tf.summary.merge_all()
writer = tf.summary.FileWriter(
            os.path.join('./tf_logs', time.strftime("%Y-%m-%d-%H-%M-%S")))
writer.add_graph(sess.graph)

with tf.device("/gpu:0"):
    sess.run(tf.global_variables_initializer())
    model.train(sess, data, train_ops,writer)
    #model.sample(sess, scores) 
    
    

epoch:  18%|█▊        | 9/50 [09:39<43:59, 64.37s/it]

----
  'doth trust.
	Romad is the gleans would die aforth, but an eunchaled grave do you contract?
ICAS	|  so fougies aloles.

LUCILIUS	Nothance I say:, despite of you, sir, let me give: yet they could so p 
----
----
 , thou easure falls in the name: o life!

	[They feed our Gracieness. Poor EGO]  So here, it is my lord, a day a wife.

CLEOPETRA	What had
	this: go, sir: you are ye before Ajax
	love no preizon in pr 
----

epoch:  28%|██▊       | 14/50 [15:02<38:42, 64.52s/it]


----
  his business, I will be forsidea.
	What haviling so rude is not most welcome; I will say there, heat me. I take thy gallic countenance and an understanding by compulsion and disgrace!
	What laby his  
----

epoch:  38%|███▊      | 19/50 [20:24<33:17, 64.43s/it]

...

epoch:  88%|████████▊ | 44/50 [47:25<06:26, 64.48s/it]


----
 ,
	Let is but blast; the winds go then takes the sooner and foil, I am bertres openly.

PAROLLES	He doth one.

	[Enter MARK ANTONAN, MENENIUS, and attendants]

KING HENRY VIII	[Within]  My lord, as he 
----

epoch:  98%|█████████▊| 49/50 [52:48<01:04, 64.52s/it]


----
  'Welro is thine honest,
	We
	had pending till the blood, and chousiddund: and this is not that there's the bottous body of the borrow'd bears: good yokhip,' your royal lady laughed to hold his gait.
 
----

epoch: 100%|██████████| 50/50 [53:52<00:00, 64.54s/it]

In [147]:
model.sample(sess, scores) 

----
 , with whipt in Rome regards vightousse.

PANDARUS	Therefore, do you right: after more thing, ha,'--this suages and smuth books;
	Have I find it to seek him once to thine eyes, that have respected wit 
----


In [ ]:
model.sample(sess, scores) 

In [120]:
[3]*3

[3, 3, 3]